In [66]:
# flake8: noqa: E501
#
# En este dataset se desea pronosticar el default (pago) del cliente el próximo
# mes a partir de 23 variables explicativas.
#
#   LIMIT_BAL: Monto del credito otorgado. Incluye el credito individual y el
#              credito familiar (suplementario).
#         SEX: Genero (1=male; 2=female).
#   EDUCATION: Educacion (0=N/A; 1=graduate school; 2=university; 3=high school; 4=others).
#    MARRIAGE: Estado civil (0=N/A; 1=married; 2=single; 3=others).
#         AGE: Edad (years).
#       PAY_0: Historia de pagos pasados. Estado del pago en septiembre, 2005.
#       PAY_2: Historia de pagos pasados. Estado del pago en agosto, 2005.
#       PAY_3: Historia de pagos pasados. Estado del pago en julio, 2005.
#       PAY_4: Historia de pagos pasados. Estado del pago en junio, 2005.
#       PAY_5: Historia de pagos pasados. Estado del pago en mayo, 2005.
#       PAY_6: Historia de pagos pasados. Estado del pago en abril, 2005.
#   BILL_AMT1: Historia de pagos pasados. Monto a pagar en septiembre, 2005.
#   BILL_AMT2: Historia de pagos pasados. Monto a pagar en agosto, 2005.
#   BILL_AMT3: Historia de pagos pasados. Monto a pagar en julio, 2005.
#   BILL_AMT4: Historia de pagos pasados. Monto a pagar en junio, 2005.
#   BILL_AMT5: Historia de pagos pasados. Monto a pagar en mayo, 2005.
#   BILL_AMT6: Historia de pagos pasados. Monto a pagar en abril, 2005.
#    PAY_AMT1: Historia de pagos pasados. Monto pagado en septiembre, 2005.
#    PAY_AMT2: Historia de pagos pasados. Monto pagado en agosto, 2005.
#    PAY_AMT3: Historia de pagos pasados. Monto pagado en julio, 2005.
#    PAY_AMT4: Historia de pagos pasados. Monto pagado en junio, 2005.
#    PAY_AMT5: Historia de pagos pasados. Monto pagado en mayo, 2005.
#    PAY_AMT6: Historia de pagos pasados. Monto pagado en abril, 2005.
#
# La variable "default payment next month" corresponde a la variable objetivo.
#
# El dataset ya se encuentra dividido en conjuntos de entrenamiento y prueba
# en la carpeta "../files/input/".
#
# Los pasos que debe seguir para la construcción de un modelo de
# clasificación están descritos a continuación.
#
#
# Paso 1.
# Realice la limpieza de los datasets:
import pandas as pd
import numpy as np

dataset_test = pd.read_csv(
    "../files/input/test_data.csv.zip",
    index_col=False,
    compression="zip",
)
dataset_train = pd.read_csv(
    "../files/input/train_data.csv.zip",
    index_col=False,
    compression="zip",
)
# - Renombre la columna "default payment next month" a "default".
dataset_test.rename(columns={"default payment next month": "default"}, inplace=True)
dataset_train.rename(columns={"default payment next month": "default"}, inplace=True)
# - Remueva la columna "ID".
dataset_test.pop("ID")
dataset_train.pop("ID")

dataset_train['EDUCATION'] = dataset_train['EDUCATION'].apply(lambda x: x if x > 0 else np.nan)
dataset_test['EDUCATION'] = dataset_test['EDUCATION'].apply(lambda x: x if x > 0 else np.nan)

dataset_train['MARRIAGE'] = dataset_train['MARRIAGE'].apply(lambda x: x if x > 0 else np.nan)
dataset_test['MARRIAGE'] = dataset_test['MARRIAGE'].apply(lambda x: x if x > 0 else np.nan)

# - Elimine los registros con informacion no disponible.
dataset_test = dataset_test.dropna()
dataset = dataset_train.dropna()
# - Para la columna EDUCATION, valores > 4 indican niveles superiores
#   de educación, agrupe estos valores en la categoría "others".
dataset_test['EDUCATION'] = dataset_test['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
dataset_train['EDUCATION'] = dataset_train['EDUCATION'].apply(lambda x: 4 if x > 4 else x)

# - Renombre la columna "default payment next month" a "default" (Repetido)
# - Remueva la columna "ID". (Repetido)
#
#


In [67]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.
#
y_train=dataset_train.copy()
y_train=y_train.pop("default")

x_train = dataset_train.copy()
x_train.pop("default")

y_test=dataset_test.copy()
y_test=y_test.pop("default")

x_test = dataset_test.copy()
x_test.pop("default")

0       1
1       0
2       0
3       0
4       0
       ..
8995    0
8996    0
8997    0
8998    1
8999    1
Name: default, Length: 8979, dtype: int64

In [68]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Escala las demas variables al intervalo [0, 1].
# - Selecciona las K mejores caracteristicas.
# - Ajusta un modelo de regresion logistica.

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

# Bloque 1: Crear un transformador personalizado para castear a entero
class CastToInteger(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            X[col] = X[col].astype(int)  # Convertir a entero
        return X

# Bloque 2: Identificar columnas categóricas y numéricas
categorical_columns = ['SEX', 'EDUCATION', 'MARRIAGE']
numeric_columns = [col for col in x_train.columns if col not in categorical_columns]

# Bloque 3: Preprocesamiento de variables categóricas
# categorical_transformer = Pipeline(steps=[
#     ('cast_to_int', CastToInteger(categorical_columns)),  # Castear columnas categóricas a entero
#     ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar valores faltantes
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))     # Aplicar One-Hot Encoding
# ])

# Bloque 4: Preprocesamiento de variables numéricas
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),  # Imputar valores faltantes
#     ('scaler', MinMaxScaler())                    # Escalar al intervalo [0, 1]
# ])

# Bloque 5: Crear el ColumnTransformer para aplicar transformaciones específicas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_columns),  # Transformar categóricas
        ('num', MinMaxScaler(), numeric_columns)           # Transformar numéricas
    ],
    remainder='passthrough'  # Mantener columnas no especificadas
)

# Bloque 6: Selección de las K mejores características
k_best_selector = SelectKBest(score_func=f_classif, k=10)

# Bloque 7: Modelo de regresión logística
logistic_model = LogisticRegression(random_state=42, max_iter=1000)

# Bloque 8: Crear el Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),     # Preprocesar datos
    ('feature_selector', k_best_selector),  # Selección de características
    ('classifier', logistic_model)      # Modelo de regresión logística
])

# Bloque 9: Entrenar el Pipeline con el conjunto de entrenamiento
pipeline.fit(x_train, y_train)

print("\nPipeline de clasificación ajustado exitosamente.")



Pipeline de clasificación ajustado exitosamente.


In [69]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score

# Paso 4: Optimización de hiperparámetros con validación cruzada

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'feature_selector__k': [20],                 # Número de características a seleccionar
    'classifier__C': [0.1],                # Regularización de la regresión logística
    'classifier__solver': ['newton-cg'],       # Solvers disponibles para regresión logística
    'classifier__penalty': ['l2'],
    'classifier__class_weight': ['balanced'],
    'classifier__max_iter': [1000]
}

# Crear un estimador GridSearchCV con validación cruzada
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='balanced_accuracy',  # Usar precisión balanceada como métrica
    cv=10,                                        # Número de particiones para la validación cruzada
    n_jobs=-1,                                    # Usar todos los núcleos disponibles
    verbose=1                                     # Nivel de detalle del proceso
)

# Ajustar el GridSearchCV con los datos de entrenamiento
grid_search.fit(x_train, y_train)

# Mostrar los mejores parámetros encontrados y la puntuación
print("\nMejores parámetros encontrados:")
print(grid_search.best_params_)
print(f"Mejor precisión balanceada (validación cruzada): {grid_search.best_score_:.4f}")

# Actualizar el pipeline con los mejores parámetros encontrados
best_pipeline = grid_search.best_estimator_


Fitting 10 folds for each of 1 candidates, totalling 10 fits

Mejores parámetros encontrados:
{'classifier__C': 0.1, 'classifier__class_weight': 'balanced', 'classifier__max_iter': 1000, 'classifier__penalty': 'l2', 'classifier__solver': 'newton-cg', 'feature_selector__k': 20}
Mejor precisión balanceada (validación cruzada): 0.6632


In [70]:
# import os
# import pickle

# dir_path = '../files/models'

# #os.makedirs(os.path.dirname(dir_path), exist_ok=True) # Otra forma de crear/verificar la direccion

# if not os.path.exists(dir_path):
#     os.makedirs(dir_path)
#     with open('../files/models/model.pkl.gz', 'wb') as f:
#         pickle.dump(grid_search, f)
# else:
#     with open('../files/models/model.pkl.gz', 'wb') as f:
#         pickle.dump(grid_search, f)

import os
import pickle
import gzip

dir_path = '../files/models'

#os.makedirs(os.path.dirname(dir_path), exist_ok=True) # Otra forma de crear/verificar la direccion

if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(grid_search, f)
else:
    with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(grid_search, f)


# import os
# import joblib

# # Paso 5: Guardar el modelo optimizado, asegurándose de que la ruta exista

# # Definir la ruta del archivo de salida
# output_dir = "../files/models"
# output_path = os.path.join(output_dir, "model.pkl")

# # Crear las carpetas necesarias si no existen
# os.makedirs(output_dir, exist_ok=True)

# # Guardar el modelo en el archivo
# joblib.dump(grid_search, output_path)
# print(f"\nModelo guardado exitosamente en '{output_path}'")

In [71]:
import os
import json
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score

# Paso 6: Calcular métricas y guardar en un archivo JSON, asegurándose de que la ruta exista

# Definir la función para calcular métricas
def calculate_metrics(y_true, y_pred, dataset_name):
    metrics = {
        'type': 'metrics',
        'dataset': dataset_name,
        'precision': precision_score(y_true, y_pred, average='binary'),
        'balanced_accuracy': balanced_accuracy_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred, average='binary'),
        'f1_score': f1_score(y_true, y_pred, average='binary'),
    }
    return metrics

# Generar predicciones para los conjuntos de entrenamiento y prueba
y_train_pred = best_pipeline.predict(x_train)
y_test_pred = best_pipeline.predict(x_test)

# Calcular métricas para cada conjunto
train_metrics = calculate_metrics(y_train, y_train_pred, 'train')
test_metrics = calculate_metrics(y_test, y_test_pred, 'test')

# Definir la ruta del archivo de salida
output_dir = "../files/output"
output_path = os.path.join(output_dir, "metrics.json")

# Crear las carpetas necesarias si no existen
os.makedirs(output_dir, exist_ok=True)

# Guardar las métricas en un archivo JSON
with open(output_path, 'w') as f:
    json.dump(train_metrics, f)
    f.write("\n")
    json.dump(test_metrics, f)


print(f"\nMétricas guardadas exitosamente en '{output_path}'")



Métricas guardadas exitosamente en '../files/output\metrics.json'


In [72]:
import os
import json
from sklearn.metrics import confusion_matrix

# Paso 7: Calcular matrices de confusión y añadirlas al archivo JSON

# Función para generar la matriz de confusión en el formato requerido
def format_confusion_matrix(cm, dataset_name):
    return {
        'type': 'cm_matrix',
        'dataset': dataset_name,
        'true_0': {
            'predicted_0': int(cm[0, 0]),
            'predicted_1': int(cm[0, 1])
        },
        'true_1': {
            'predicted_0': int(cm[1, 0]),
            'predicted_1': int(cm[1, 1])
        }
    }

# Calcular las matrices de confusión
cm_train = confusion_matrix(y_train, y_train_pred)
cm_test = confusion_matrix(y_test, y_test_pred)

# Formatear las matrices de confusión
train_cm_metrics = format_confusion_matrix(cm_train, 'train')
test_cm_metrics = format_confusion_matrix(cm_test, 'test')

# Definir la ruta del archivo para las matrices de confusión
confusion_matrix_dir = "../files/output"
confusion_matrix_path = os.path.join(confusion_matrix_dir, "metrics.json")

# Crear las carpetas necesarias si no existen
os.makedirs(confusion_matrix_dir, exist_ok=True)

# Cargar métricas existentes o inicializar una lista vacía si el archivo está vacío o no existe
metrics = []
if os.path.exists(confusion_matrix_path):
    try:
        with open(confusion_matrix_path, 'r') as f:
            content = f.read().strip()  # Leer contenido del archivo
            if content:  # Validar que no esté vacío
                metrics = json.loads(content)
    except (json.JSONDecodeError, ValueError):
        print(f"Advertencia: El archivo '{confusion_matrix_path}' tiene un formato inválido. Será sobrescrito.")

# Añadir las matrices de confusión al archivo


# Guardar el archivo actualizado
with open(confusion_matrix_path, 'a') as f:
    f.write("\n")
    json.dump(train_cm_metrics, f)
    f.write("\n")
    json.dump(test_cm_metrics, f)

print(f"\nMatrices de confusión guardadas exitosamente en '{confusion_matrix_path}'")


Advertencia: El archivo '../files/output\metrics.json' tiene un formato inválido. Será sobrescrito.

Matrices de confusión guardadas exitosamente en '../files/output\metrics.json'
